In [ ]:
import os

In [ ]:
def get_filePath_list(file_dir):
    filePath_list = []
    for walk in os.walk(file_dir):
        part_filePath_list = [os.path.join(walk[0], file) for file in walk[2]]
        filePath_list.extend(part_filePath_list)
    return filePath_list

def get_files_list(file_dir, postfix='ALL'):
    postfix = postfix.split('.')[-1]
    file_list = []
    filePath_list = get_filePath_list(file_dir)
    if postfix == 'ALL':
        file_list = filePath_list
    else:
        for file in filePath_list:
            basename=os.path.basename(file)
            postfix_basename = basename.split('.')[-1]
            if postfix_basename == postfix:
                file_list.append(file)
    file_list.sort()
    return file_list

In [ ]:
import jieba

In [ ]:
def segment_files(file_list, segment_out_dir, stopwords=[]):
    for i, file in enumerate(file_list):
        segment_out_name = os.path.join(segment_out_dir, 'segment_{}.txt'.format(i))
        with open(file, 'rb') as fin:
            document = fin.read()
            document_cut = jieba.cut(document)
            sentence_segment=[]
            for word in document_cut:
                if word not in stopwords:
                    sentence_segment.append(word)
            result = ' '.join(sentence_segment)
            result = result.encode('utf-8')
            with open(segment_out_name, 'wb') as fout:
                fout.write(result)        

In [ ]:
#source and segment dir
source_folder = './three_kingdoms/source'
segment_folder = './three_kingdoms/segment'

file_list = get_files_list(source_folder, postfix='*.txt')
segment_files(file_list, segment_folder)

In [ ]:
from gensim.models import word2vec
import multiprocessing

In [ ]:
sentences = word2vec.PathLineSentences(segment_folder)

In [ ]:
for i, sentence in enumerate(sentences):
    if i < 100:
        print(sentence)

In [ ]:
print('Train word2vec model with {} CPUs'.format(multiprocessing.cpu_count()))
model = word2vec.Word2Vec(sentences, size=128, window=5, min_count = 5, workers=multiprocessing.cpu_count())

if not os.path.exists('models'):
    try:
        os.makedirs('models')
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
# save model
model.save('./models/word2Vec.model')

In [ ]:
print(model.wv.most_similar('曹操'))
print(model.wv.most_similar(positive=['曹操', '刘备'], negative=['张飞']))